# 量子アニーリングマシン・イジングマシンによるピクロスの解法

本サンプルコードでは、量子アニーリングマシン・イジングマシンによるパズルゲーム、ピクロスの解法について解説し、Amplifyを用いて実装します。本ノートブックは、以下の構成となっています。

- 1\. [ピクロスのルールと解法の道筋](#1)
  - 1.1\. [ピクロスのルール](#1_1)
  - 1.2\. [ピクロス解法の概略](#1_2)
  - 1.3\. [語句の定義と塗り方の表現](#1_3)
    - 1.3.1\. [黒の先頭、黒の最後尾](#1_3_1)
    - 1.3.2\. [マスの状態](#1_3_2)
    - 1.3.3\. [塗り方の行列表現](#1_3_3)
- 2\. [ピクロス解法の定式化](#2)
  - 2.1\. [初期値制約](#2_1)
  - 2.2\. [遷移制約](#2_2)
  - 2.3\. [一致制約](#2_3)
  - 2.4\. [マス色と状態をつなげる制約](#2_4)
- 3\. [ピクロス解法の実装](#3)
  - 3.1\. [ピクロス行列のプロット関数](#3_1)
  - 3.2\. [制約式の実装](#3_2)
    - 3.2.1\. [決定変数の定義](#3_2_1)
    - 3.2.2\. [初期値制約の実装](#3_2_2)
    - 3.2.3\. [遷移制約の実装](#3_2_3)
    - 3.2.4\. [一致制約の実装](#3_2_4)
    - 3.2.5\. [マス色と状態をつなげる制約](#3_2_5)
  - 3.3\. [Amplify クライアントの設定](#3_3)
  - 3.4\. [ピクロス求解の実施](#3_4)
- 発展：[高度なチューニング](#4)

<a id="1"></a>
## 1\. ピクロスのルールと解法の道筋

<a id="1_1"></a>
### 1.1\. ピクロスのルール

ピクロスとは、上と左に数字で与えられるヒントを元にマスを塗っていき、絵を完成させるパズルです。

例えばヒントに `5` と書かれていた場合、その列の連続する5マスを黒く塗りつぶすことを表します。`1 1 1` のように複数の数字が書かれていた場合、間に1つ以上の空白を挟んで黒く塗りつぶすことを表します。また、`1 3` の場合は、1つのマスと連続する3つのマスを黒く塗り、その間に1つ以上の空白を挟む、という塗り方を表します。

以下はヒントに従って塗りつぶされたマスの一例です。

![5x5_solution](../figures/picross-puzzle/5x5_solution.png)


<a id="1_2"></a>
### 1.2\. ピクロス解法の概略

ピクロスが完成するとは、上と左のヒントが同時に満たされるようにマスが塗られることを指します。

あるピクロスの問題を左のヒントのみを満たすように塗った場合、塗り方は一つに定まりません。例えば、以下のような異なる塗り方が考えられます。

![5x5_solution_left_1](../figures/picross-puzzle/5x5_solution_left_1.png)
![5x5_solution_left_2](../figures/picross-puzzle/5x5_solution_left_2.png)

このような左のヒントのみを満たすような塗り方を「 **塗り方0** 」とします。

同様に上のヒントのみを満たすような塗り方を「 **塗り方1** 」とします。

**塗り方0** による各マスの白黒が **塗り方1** と一致していたとき、上と左のヒントが同時に満たされてピクロスパズルが完成します。

したがって、

* **条件 (i)** ・・・ **塗り方0** と **塗り方1** が一致している、
* **条件 (ii)**・・・縦横各列の塗り方それぞれがヒントを満たしている

の二つの条件を満足するような塗り方をイジングマシン・量子アニーリングで探索することでピクロスパズルを解いていきます。

<a id="1_3"></a>
### 1.3\. 語句の定義と塗り方の表現

<a id="1_3_1"></a>
#### 1.3.1\. 黒の先頭、黒の最後尾

次のように、ピクロスパズルから1列のみを取り出します。

![single_row_2_3](../figures/picross-puzzle/single_row_2_3.png)

ここで、連続する黒のマスについて、 **黒の先頭** と **黒の最後尾** を以下のように定義します。

- **黒の先頭** とは、前のマスが存在するなら、前のマスは白で塗られなければならない黒である。（下図橙枠）
- **黒の最後尾** とは、後ろのマスが存在するなら、後ろのマスは白で塗られなければならない黒である。（下図緑枠）

![single_row_2_3_highlighted](../figures/picross-puzzle/single_row_2_3_highlighted.png)

これらの語句を使うと、前節で紹介した **条件 (ii)** は、1つ前のマスから次のマスへの遷移という部分に着目して、次のような制約として考えることができます。

- **(ii-a)** 各マスは必ず黒か白で塗られる
- **(ii-b)** 白の次のマスは白か **黒の先頭**
- **(ii-c)** 白の1つ前のマスは白か **黒の最後尾**
- **(ii-d)** **黒の最後尾** を除く黒の次のマスは黒

以上のような制約条件を適切に定義することで、イジングマシン・量子アニーリングを使ってピクロスを完成させることができます。

<a id="1_3_2"></a>
#### 1.3.2\. マスの状態

今、各マスを白か黒の二種類で表現していますが、制約条件の定式化を可能にするためにより細かい状態を定義します。

- 黒マスを、何番目に出現した黒マスかによって状態を定義する  
例えば、ヒントが `2 3` と与えられていた場合、黒マスの状態は以下のように定義されます。

  ![single_row_2_3_black_state_id](../figures/picross-puzzle/single_row_2_3_black_state_id.png)

- 白マスを、それまでに出現した黒マスの数によって状態を定義する  
例えばヒントが `2 3` と与えられていた場合、白マスの状態は以下のように定義されます。  

  ![single_row_2_3_white_state_id](../figures/picross-puzzle/single_row_2_3_white_state_id.png)

- 定義した状態を出現順に統合する  
例えばヒントが `2 3` と与えられていた場合、各マスの状態は以下のように定義されます。  

  ![single_row_2_3_black_white_state_id](../figures/picross-puzzle/single_row_2_3_black_white_state_id.png)  

  このように各マスの状態を定義し、各ピクロス列の $i$ 番目のヒントを $h_i$ と表記すると、状態の数 $W=1+\sum_i (h_i+1)$ と記述することができます。例えばヒントとして `2 3` が与えられている上記のピクロス列では、$h_1=2, h_2=3$ であり、$W=8$ が状態の数として得られます。

<a id="1_3_3"></a>
#### 1.3.3\. 塗り方の行列表現

例えば、`2 3` のヒントのピクロス列に対して以下のような塗り方を試行するとします。

![single_row_2_3_black_white_state_id](../figures/picross-puzzle/single_row_2_3_black_white_state_id.png)

この塗り方を体系的に表現するために、次のような方法が考えられます。

- $i$ マス目の色を次の表のように表すことができます。

<div align="center">

表1：$i$ マス目の色

| $i$ マス目 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 |
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
| **色** | 白 | 白 | 黒 | 黒 | 白 | 白 | 黒 | 黒 | 黒 | 白 |

</div>

- 二値変数行列を用いると、マス $i$ の色と状態を表現することもできます。

<div align="center">

表2：マス $i$ の色と状態

| $i$ マス目 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 |
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
| **白（状態①）** | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| **黒（状態②）** | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| **黒（状態③）** | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 |
| **白（状態④）** | 0 | 0 | 0 | 0 | 1 | 1 | 0 | 0 | 0 | 0 |
| **黒（状態⑤）** | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 0 | 0 | 0 |
| **黒（状態⑥）** | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 0 | 0 |
| **黒（状態⑦）** | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 0 |
| **白（状態⑧）** | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 1 |

</div>

以上のように特定の塗り方に対して、ピクロス列の色と状態を行列で表現することができました。

<a id="2"></a>
## 2\. ピクロス解法の定式化

1節で説明されたピクロスの解法に必要な条件を具体的に定式化していきます。定式化に現れる定数及び変数は以下の通りです（既に紹介した定数・変数も含む）。

**定数**
- $V$：ピクロス行列の縦のマス数
- $H$：ピクロス行列の横のマス数
- $N$：一列のマス数
- $h_i$：$i$ 番目のヒントの数字
- $W$：状態数 ($= 1 + \sum_i (h_i+1)$)
- $s_i$：状態$i$ ($i \in \{1, \dots, W\}$)
- $S_{\rm{white}}, S_{\rm{black}}$：白の状態の集合、黒の状態の集合 ($s_i$ は必ず $S_{\rm{white}}$ または $S_{\rm{black}}$ のいずれかに属する)

**変数**
- $q^{(m)}_{i,j} \in \{0, 1\} \quad (i \in \{1, \dots, V\})$ $\quad (j \in \{1, \dots, H\})$  
  
  $i$ 行 $j$ 列目のマスの色（表1参照）。$q^{(0)}$ は左のヒントのみを満たす **塗り方0** 、$q^{(1)}$ は上のヒントのみを満たす **塗り方1** を示します。$i$ 行 $j$ 列目のマスが黒なら $q_{i,j}=1$、白なら $q_{i,j}=0$ です。
- $p_{i,j} \in \{0, 1\} \quad (i \in \{1, \dots, N\}) \quad (j \in \{1, \dots, W\})$  
  
  マス $i$ の状態（表2参照）。マス $i$ の状態が $j$ のとき $p_{i,j}=1$、そうでない場合は $p_{i,j}=0$。

これらの定数と変数に基づき、各制約の定式化を行っていきます。

<a id="2_1"></a>
### 2.1\. 初期値制約

ピクロスを解くには、各行・列に対して様々な状態を探索します。つまり、探索中には、1つのピクロス列に対して、様々な状態に対応する二値変数行列（例：表2）を評価することになります。

![single_row_2_3_black_white_state_id](../figures/picross-puzzle/single_row_2_3_black_white_state_id.png)

ここで、各マス $i$ は任意の状態 $j$ をとることはできません。例えば、`2 3` のヒントが与えられた上のピクロス列の例では、取りうる状態数は、$W=1 + \sum_i (h_i+1)=8$ 個であり、一番左のマス ($i=1$) は、

- 状態①（列の最初が白で始まる場合）、または
- 状態②（列の最初が **黒の先頭** で始まり、状態①白は存在しない場合）

のみ取り得ることが分かります。マス ($i=1$) が状態③や④を取ることはありません。

同様に、下図のように、左から2番目のマス ($i=2$) は、状態①や状態②、状態③を取り得ることが分かります。逆に、それより大きな状態を取ることはありません。

![single_row_2nd_left_state](../figures/picross-puzzle/single_row_2nd_left_state.png)

このように、マス $i$ が決して取り得ない状態を考慮し、二値変数行列を予めゼロ埋めすることができます。例えば、上のピクロス列では、次のようにゼロ埋めが可能です。

<div align="center">

表3：ゼロ埋め後のマス $i$ の色と状態

| $i$ マス目 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 |
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
| **白（状態①）** |  |  |  |  | 0 | 0 | 0 | 0 | 0 | 0 |
| **黒（状態②）** |  |  |  |  |  | 0 | 0 | 0 | 0 | 0 |
| **黒（状態③）** | 0 |  |  |  |  |  | 0 | 0 | 0 | 0 |
| **白（状態④）** | 0 | 0 |  |  |  |  |  | 0 | 0 | 0 |
| **黒（状態⑤）** | 0 | 0 | 0 |  |  |  |  |  | 0 | 0 |
| **黒（状態⑥）** | 0 | 0 | 0 | 0 |  |  |  |  |  | 0 |
| **黒（状態⑦）** | 0 | 0 | 0 | 0 | 0 |  |  |  |  |  |
| **白（状態⑧）** | 0 | 0 | 0 | 0 | 0 | 0 |  |  |  |  |

</div>

このように、ピクロス列の状態の取り方を制限することができ、解法における探索範囲を限定することができます。ここではこの制約を初期値制約と呼び、マスの状態を表す二値変数行列をゼロ埋めすることで実現します。初期値制約の定式化は、マス $i$ の状態を表す変数 $p_{i,j}$ を使い、次のように実施することができます。

$$
\left\{
    \begin{align*}
        &\begin{split}
            &p_{i, j} = 0 \quad (j > i+1)
        \end{split}\\
        &\begin{split}
            &p_{i, j} = 0 \quad (W-j > N-i+1)
        \end{split}\\
    \end{align*}
\right.
$$

ここで、$i \in \{1, ... , N\}, j \in \{1, ... , W\}$ です。$N = 5$, $W=8$ である場合、表3のように初期値制約できるか確認してみてください。

<a id="2_2"></a>
### 2.2\. 遷移制約

遷移制約は既に、[1.3.1\. 黒の先頭、黒の最後尾](#1_3_1)において **(ii-a)**～**(ii-d)** として紹介しました。本節では、この遷移制約を定式化します。前述の以下のピクロス列の例を用いて解説します。

![single_row_2_3_black_white_state_id](../figures/picross-puzzle/single_row_2_3_black_white_state_id.png)

各マスの状態を用いると、既に紹介した1つ前のマスから次のマスへの遷移の条件 **(ii-a)**～**(ii-d)** は、次のように、より詳細に記述することができます。

- **(ii-a')** 各マスはそれぞれ一つの状態をとる

- **(ii-b')** $i$ マス目が状態 $j$ で、状態 $j$ が白のとき、$i+1$ マス目は状態 $j$（白）または状態 $j+1$（ **黒の先頭** ）  
  （例）$i$ マス目が状態④（白）のとき、$i+1$ マス目は状態④（白）または状態⑤（ **黒の先頭** ）となります。

- **(ii-c')** $i$ マス目が状態 $j$ で、状態 $j$ が白のとき、$i−1$ マス目は状態 $j$（白）または状態 $j-1$（ **黒の最後尾** ）  
  （例）$i$ マス目が状態④（白）のとき、$i-1$ マス目は状態④（白）または状態③（ **黒の最後尾** ）となります。

- **(ii-d')** $i$マス目の状態が黒 $j$（$\neq$ **黒の最後尾** ）のとき、$i+1$ マス目の状態は黒 $j+1$  
  （例）$i$ マス目が状態⑤（黒）のとき、$i+1$ マス目は状態⑥（黒）となります。

これらの遷移制約に対応して、$i \in \{1, ... , N\}$ に対して以下が成り立ちます。

- **(ii-a')** $\quad \sum_j p_{i,j}=1$

- **(ii-b')** $\quad p_{i, j} \land (1-p_{i+1, j}) = p_{i+1, j+1} \quad (\forall j \neq W, s_j \in S_{\rm{white}})$

- **(ii-c')** $\quad p_{i, j} \land (1-p_{i-1, j}) = p_{i-1, j-1} \quad (\forall j \neq 1, s_j \in S_{\rm{white}})$

- **(ii-d')** $\quad p_{i, j} = p_{i+1, j+1} \quad (\forall j, \{s_j, s_{j+1}\} \subset S_{\rm{black}})$

<a id="2_3"></a>
### 2.3\. 一致制約

既に、[1.2\. ピクロス解法の概略](#1_2) で紹介した、

> **条件 (i)** 『 **塗り方0** と **塗り方1** が一致している』

は、より明確に記述すると、任意のピクロス行列のマス $i,j$ において、 **塗り方0** の $(i,j)$ マスと **塗り方1** の $(i,j)$ マスは、両方黒である、または両方白である、ということになります。これを定式化すると、$i \in \{1, ... , V\}, j \in \{1, ... , H\}$に対して、

- **(i')** $\quad q^{(0)}_{i,j} = q^{(1)}_{i,j}$

と記述することができます。

<a id="2_4"></a>
### 2.4\. マス色と状態をつなげる制約

遷移制約では、1つのピクロス列の制約について定義しました。実際のピクロスでは、縦列が $V$ 個、横列が $H$ 個あるので、$H+V$ 列分に対して定式化を行う必要があります。そこで変数に上の添え字を追加し、上のヒントか左のヒントか、何列目を指しているかの二つの情報を持たせます。具体的には、添え字を $(a,b)$ とし、$a$ が $0$ のときは左のヒント、$1$ のとき上のヒントを示します。また、$b$ は上または左から何列目なのかを示します。

これらの表記を使い、$q^{(m)}_{i,j}$（$i$ 行 $j$ 列目のマスの色、塗り方 $m$）と、あるピクロス列 $(a,b)$における $p_{i,k}^{(a,b)}$（マス $i$ の状態 $k$）の関係に関する制約条件を次のように定式化することができます。

- **制約 (iii)**
$$
\begin{align*}
    \sum_{k| s^{(0, j)}_k \in S^{(0, j)}_{\rm{black}}} p^{(0, j)}_{i, k} = q^{(0)}_{i,j}
\end{align*}
$$

$$
\begin{align*}
    \sum_{k| s^{(1, i)}_k \in S^{(1, i)}_{\rm{black}}} p^{(1, i)}_{j, k} = q^{(1)}_{i,j}
\end{align*}
$$



ただし、$i \in \{1, ... , H\}, j \in \{1, ... , V\}$。

<a id="3"></a>
## 3\. ピクロス解法の実装

それでは、これまで行ってきた制約を基に、ピクロス解法を Amplify を使って実装しましょう。

<a id="3_1"></a>
### 3.1\. ピクロス行列のプロット関数

まず、ピクロス行列及び解を表示するための関数 `plot_solution` を定義します。

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_solution(v_hints, h_hints, solution=np.zeros((0, 0))):
    if solution.shape[0] == len(v_hints) and solution.shape[1] == len(h_hints):
        solution = solution
    else:
        solution = np.zeros((len(h_hints), len(v_hints)))

    _, ax = plt.subplots()
    ax.tick_params(
        which="both",
        top=True,
        bottom=False,
        labeltop=True,
        labelbottom=False,
        length=0,
    )
    ax.tick_params(axis="x")

    ax.imshow(solution, cmap="Greys", aspect="equal")
    # 主目盛り
    ax.set_xticks(np.arange(len(h_hints)))
    ax.set_yticks(np.arange(len(v_hints)))
    # 副目盛り
    ax.set_xticks(np.arange(-0.5, len(h_hints), 1), minor=True)
    ax.set_yticks(np.arange(-0.5, len(v_hints), 1), minor=True)
    # 主目盛りのラベル
    ax.set_xticklabels(["\n".join(map(str, hint)) for hint in h_hints])
    ax.set_yticklabels(["  ".join(map(str, hint)) for hint in v_hints])
    ax.set_xlim([-0.5, len(h_hints) - 0.5])
    ax.set_ylim([len(v_hints) - 0.5, -0.5])
    ax.set_title(f"{len(h_hints)} x {len(v_hints)}", fontsize=20, pad=20)
    # 副目盛りに基づく格子
    ax.grid(which="minor", color="#aaaaaa", linestyle="-", linewidth=1)
    return plt.show()

また、定義した `plot_solution` を用いて、[1.1\. ピクロスのルール](#1_1) で紹介したピクロスパズルと同じ問題を作成し、描画します。

これで、ピクロス解法実装の下準備が整いました。

In [ ]:
# ヒントをリストで表記
# 左のヒント
left_hints = [[5], [1], [5], [1], [5]]
# 上のヒント
top_hints = [[3, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 3]]

# ピクロス行列の縦、横サイズ
v_len, h_len = len(left_hints), len(top_hints)

# ピクロス問題をプロット
plot_solution(left_hints, top_hints, np.zeros((v_len, h_len)))

<a id="3_2"></a>
### 3.2\. 制約式の実装

<a id="3_2_1"></a>
#### 3.2.1\. 決定変数の定義

次に、必要な決定変数を定義します。Amplify の `BinarySymbolGenerator` を用います。

ピクロス行列内 $i,j$ に位置するマスの色を表す二次元配列 `q` を定義します。[2\. ピクロス解法の定式化](#2)で紹介した $q^{(m)}_{i,j}$ 同様、左と上のヒントそれぞれで配列を定義します。値は、0: 白、1: 黒に対応します。

In [ ]:
from amplify import BinarySymbolGenerator

gen = BinarySymbolGenerator()

# 左のヒントに基づくピクロス行列、マス (i,j) の色
q_from_left_hints = gen.array(v_len, h_len)
# 上のヒントに基づくピクロス行列、マス (i,j) の色
q_from_top_hints = gen.array(h_len, v_len)

さらに各ピクロス列ごとに、（マスの数×状態の数）となる二次元配列 `p` を定義します。各マスには一つの状態が割り当てられます。`p` は、[2.4\. マス色と状態をつなげる制約](#2_4)で紹介した $p_{i,k}^{(a,b)}$ 同様、左のヒント  `left_hints`  と上のヒント `top_hints` それぞれで配列を定義します。

In [ ]:
# 左と上のヒントそれぞれに対応する変数を作成する
# (マスの数, 状態数)の二次元配列
p_from_left_hints = [gen.array(h_len, sum(hint) + len(hint) + 1) for hint in left_hints]
p_from_top_hints = [gen.array(v_len, sum(hint) + len(hint) + 1) for hint in top_hints]

<a id="3_2_2"></a>
#### 3.2.2\. 初期値制約の実装

[2.1\. 初期値制約](#2_1)で解説した初期値制約に関して、次のように二値変数行列のゼロ埋めを実装します。

$$
\left\{
    \begin{align*}
        &\begin{split}
            &p_{i, j} = 0 \quad (j > i+1)
        \end{split}\\
        &\begin{split}
            &p_{i, j} = 0 \quad (W-j > N-i+1)
        \end{split}\\
    \end{align*}
\right.
$$

In [ ]:
def assign_init_value(p):
    n = p.shape[0]
    for i in range(n):
        # 初期値制約：各マスが取り得ない状態を 0 埋め
        p[i, i + 2 :] = 0
        p[-i - 1, : -i - 2] = 0

<a id="3_2_3"></a>
#### 3.2.3\. 遷移制約の実装

遷移制約は、[2.2\. 遷移制約](#2_2)で紹介しましたが、全部で4つの制約条件から構成されます。これらを一つ一つ実装します。

##### **(ii-a')**

> 各マスはそれぞれ一つの状態をとる

$$
\quad \sum_j p_{i,j}=1
$$

In [ ]:
from amplify.constraint import equal_to
from amplify import sum_poly


def make_state_constraint(p):
    state_constraints = []
    # 遷移制約(ii-a')：各マスはそれぞれ一つの状態をとる
    n = p.shape[0]
    w = p.shape[1]
    for j in range(n):
        c = sum_poly(w, lambda k: p[j, k])
        state_constraints.append(equal_to(c, 1))
    return state_constraints

##### **(ii-b')**
> $i$ マス目が状態 $j$ で、状態 $j$ が白のとき、$i+1$ マス目は状態 $j$（白）または状態 $j+1$（ **黒の先頭** ）

$$
\quad p_{i, j} \land (1-p_{i+1, j}) = p_{i+1, j+1} \quad (\forall j \neq W, s_j \in S_{\rm{white}})
$$

この制約式を、Amplify の[ペナルティ関数](https://amplify.fixstars.com/ja/docs/constraint.html#id12)を使って実装します。まず、上式の左辺を右辺に移項し、両辺を2乗します。論理積 $\land$ は、乗算に変換できます。この右辺を本制約に関するペナルティ $P_b$ として、

$$
\quad P_b = \left[ p_{i, j} (1-p_{i+1, j}) - p_{i+1, j+1}\right]^2 \quad (\forall j \neq W, s_j \in S_{\rm{white}})
$$

とします。$P_b=0$ である $(p_{i,j}, p_{i+1,j}, p_{i+1,j+1})$ の組のみ、本制約を満たすことが分かります。

上式を展開すると

$$
P_b = -p_{i,j}p_{i+1,j} - 2p_{i,j}p_{i+1,j+1} + 2\:{\color{red} p_{i,j}} \:p_{i+1,j}p_{i+1,j+1} + p_{i,j} + p_{i+1,j+1} \quad (\forall j \neq W, s_j \in S_{\rm{white}})
$$

ここで、上式はペナルティとして取り扱う式であり、$P_b \neq0$ である $(p_{i,j}, p_{i+1,j}, p_{i+1,j+1})$ の組に対しては、$P_b \neq0$ である限り、必ずしも $P_b$ を正確に計算する必要はありません。この考えに基づくと、上式で朱字になっている項を省略することができ、[補助変数の導入が必要な三次の項](https://amplify.fixstars.com/ja/docs/model.html#id2)の利用を回避できます。

最終的に、$j\neq W$ の場合の $P_b$ は、

$$
P_b = -p_{i,j}p_{i+1,j} - 2p_{i,j}p_{i+1,j+1} + 2p_{i+1,j}p_{i+1,j+1} + p_{i,j} + p_{i+1,j+1} \quad (\forall j \neq W, s_j \in S_{\rm{white}})
$$

とすることができます。また、$j=W$ の場合、上式は、

$$
P_b = -p_{i,j}p_{i+1,j} + p_{i,j} \quad (j=W, s_j \in S_{\rm{white}})
$$

となります。この2式を白色のマスに関するペナルティ関数（できるだけ0に近い値を取らせるようにする）として、次のように実装します。

In [ ]:
from amplify.constraint import penalty


def make_white_constraint_forward(p, hint):
    white_constraints = []

    # 白である状態のindexを保管する
    s_white_indexes = {0}
    cursor = 0
    for h in hint:
        s_white_indexes.add(cursor + 1 + h)
        cursor = cursor + 1 + h

    # 遷移制約(ii-b'): iマス目の状態が空白kのとき、i+1マス目の状態は空白または黒の先頭
    n = p.shape[0]
    w = p.shape[1]
    for i in range(n - 1):
        for j in s_white_indexes:
            if j == w - 1:
                c = -p[i, j] * p[i + 1, j] + p[i, j]
            else:
                c = (
                    -p[i, j] * p[i + 1, j]
                    - 2 * p[i, j] * p[i + 1, j + 1]
                    + 2 * p[i + 1, j] * p[i + 1, j + 1]
                    + p[i, j]
                    + p[i + 1, j + 1]
                )
            white_constraints.append(penalty(c))

    return white_constraints

##### **(ii-c')** 

>$i$ マス目が状態 $j$ で、状態 $j$ が白のとき、$i−1$ マス目は状態 $j$（白）または状態 $j-1$（ **黒の最後尾** ）  

$$
\quad p_{i, j} \land (1-p_{i-1, j}) = p_{i-1, j-1} \quad (\forall j \neq 1, s_j \in S_{\rm{white}})
$$

本遷移制約も、前述 **(ii-b')** と同様に式を展開し、Amplify のペナルティ関数を用いて実装します。

従って、本遷移制約のペナルティ $P_c$ は、

$$
P_c = -p_{i,j}p_{i-1,j} - 2p_{i,j}p_{i-1,j-1} + 2p_{i-1,j}p_{i-1,j-1} + p_{i,j} + p_{i-1,j-1}  \quad (\forall j \neq 1, s_j \in S_{\rm{white}})
$$

$$
P_c = -p_{i,j}p_{i-1,j} + p_{i,j}  \quad (j=1, s_j \in S_{\rm{white}})
$$


In [ ]:
def make_white_constraint_backward(p, hint):
    last_black_constraints = []

    # 白である状態のindexを保管する
    s_white_indexes = {0}
    cursor = 0
    for h in hint:
        s_white_indexes.add(cursor + 1 + h)
        cursor = cursor + 1 + h

    # 遷移制約(ii-c'): iマス目の状態が空白kのとき、i-1マス目の状態は空白または黒の最後尾
    n = p.shape[0]
    for i in range(1, n):
        for j in s_white_indexes:
            if j == 0:
                c = -p[i, j] * p[i - 1, j] + p[i, j]
            else:
                c = (
                    -p[i, j] * p[i - 1, j]
                    - 2 * p[i, j] * p[i - 1, j - 1]
                    + 2 * p[i - 1, j] * p[i - 1, j - 1]
                    + p[i, j]
                    + p[i - 1, j - 1]
                )
            last_black_constraints.append(penalty(c))

    return last_black_constraints


##### **(ii-d')** 

> $i$マス目の状態が黒 $j$（$\neq$ **黒の最後尾**）のとき、$i+1$ マス目の状態は黒 $j+1$

$$
\quad p_{i, j} = p_{i+1, j+1} \quad (\forall j, \{s_j, s_{j+1}\} \subset S_{\rm{black}})
$$

In [ ]:
def assign_black_forward(p, hint):
    # 黒である状態のindexを保管する
    s_black_indexes = set()
    cursor = 0
    for h in hint:
        s_black_indexes |= set(range(cursor + 1, cursor + 1 + h))
        cursor += h + 1

    # 遷移制約(ii-d'): iマス目の状態が黒j(≠最後尾)のとき、i+1マス目の状態は黒j+1
    n = p.shape[0]
    w = p.shape[1]
    for i in range(n - 2, -1, -1):
        for j in range(w - 2, -1, -1):
            if j in s_black_indexes and j + 1 in s_black_indexes:
                p[i, j] = p[i + 1, j + 1]

<a id="3_2_4"></a>
#### 3.2.4\. 一致制約の実装

[2.3\. 一致制約](#2_3)で紹介した、左のヒントに基づく **塗り方0** と上のヒントに基づく **塗り方1** の一致制約

$$
\quad q^{(0)}_{i,j} - q^{(1)}_{i,j} = 0
$$

は次のように実装します。

In [ ]:
def equal_constraints(q_left, q_top, v_len, h_len):
    constraints = []
    for y in range(v_len):
        for x in range(h_len):
            # 「塗り方0」と「塗り方1」の一致制約
            c = q_left[y, x] - q_top[x, y]
            constraints.append(equal_to(c, 0))

    return sum(constraints)

<a id="3_2_5"></a>
#### 3.2.5\. マス色と状態をつなげる制約

[2.4\. マス色と状態をつなげる制約](#2_4)で紹介した、制約条件は以下の式で表されます。

- **制約 (iii)**
$$
\begin{align*}
    \sum_{k| s^{(0, j)}_k \in S^{(0, j)}_{\rm{black}}} p^{(0, j)}_{i, k} = q^{(0)}_{i,j}
\end{align*}
$$

$$
\begin{align*}
    \sum_{k| s^{(1, i)}_k \in S^{(1, i)}_{\rm{black}}} p^{(1, i)}_{j, k} = q^{(1)}_{i,j}
\end{align*}
$$

ここで、実装上は、変数の数は小さいほうがより高速にイジングマシン・量子アニーリングによる求解が可能です。ピクロスにおいて、ピクロス行列のサイズが十分大きな場合、白である状態の数の方が、黒である状態の数より多いと考えられるので、参照する変数の数を減らすために以下の式に変換します。（上の式と同義）

- **制約 (iii')**
$$
\begin{align*}
    \sum_{k| s^{(0, j)}_k \in S^{(0, j)}_{\rm{white}}} p^{(0, j)}_{i, k} = 1-q^{(0)}_{i,j}
\end{align*}
$$

$$
\begin{align*}
    \sum_{k| s^{(1, i)}_k \in S^{(1, i)}_{\rm{white}}} p^{(1, i)}_{j, k} = 1-q^{(1)}_{i,j}
\end{align*}
$$

In [ ]:
def assign_state_color_link(q, p, hint):
    n = q.shape[0]
    for j in range(n):
        # 白である状態のindexを保管する
        s_white_indexes = {0}
        cursor = 0
        for h in hint:
            s_white_indexes.add(cursor + 1 + h)
            cursor = cursor + 1 + h

        # マス色と状態を繋げる制約条件 (iii')
        eq = 1 - sum_poly(s_white_indexes, lambda k: p[j, k])
        q[j] = eq

<a id="3_3"></a>
### 3.3\. Amplify クライアントの設定

Amplify のイジングマシン・量子アニーリングのクライアントを作成します。

In [ ]:
from amplify.client import FixstarsClient

client = FixstarsClient()
client.parameters.timeout = 1000  # タイムアウト1秒
# client.token = "API トークンを入力してください"

<a id="3_4"></a>
### 3.4\. ピクロス求解の実施

まず、上記で実装したすべての制約条件を足し合わせます。

In [ ]:
# 遷移制約：(ii-a')、(ii-b')、(ii-c')
state_constraints = []
white_constraints = []
last_black_constraints = []
for p, hint in zip(p_from_left_hints, left_hints):
    state_constraints += make_state_constraint(p)
    white_constraints += make_white_constraint_forward(p, hint)
    last_black_constraints += make_white_constraint_backward(p, hint)
left_constraints = (
    sum(state_constraints) + sum(white_constraints) + sum(last_black_constraints)
)

state_constraints = []
white_constraints = []
last_black_constraints = []
for p, hint in zip(p_from_top_hints, top_hints):
    state_constraints += make_state_constraint(p)
    white_constraints += make_white_constraint_forward(p, hint)
    last_black_constraints += make_white_constraint_backward(p, hint)
top_constraints = (
    sum(state_constraints) + sum(white_constraints) + sum(last_black_constraints)
)

# 左のヒントに対するピクロス列
for q, p, hint in zip(q_from_left_hints, p_from_left_hints, left_hints):
    # 初期値制約
    assign_init_value(p)
    # 遷移制約：(ii-d')
    assign_black_forward(p, hint)
    # マス色と状態をつなげる制約：(iii')
    assign_state_color_link(q, p, hint)

# 上のヒントに対するピクロス列
for q, p, hint in zip(q_from_top_hints, p_from_top_hints, top_hints):
    # 初期値制約
    assign_init_value(p)
    # 遷移制約：(ii-d')
    assign_black_forward(p, hint)
    # マス色と状態をつなげる制約：(iii')
    assign_state_color_link(q, p, hint)

# 一致制約
eq_constraints = equal_constraints(q_from_left_hints, q_from_top_hints, v_len, h_len)

# 全制約条件の足し合わせ
constraints = left_constraints + top_constraints + eq_constraints * 2

この全制約条件の足し合わせにおいて、`eq_constraints` のみ2倍しているのは、次のような理由になります。

`left_constraints` 及び `top_constraints` のそれぞれは、遷移制約(ii-a')、(ii-b')、(ii-c') を考慮しています。つまり、遷移制約(ii-a')、(ii-b')、(ii-c') は左のヒントに基づく場合と上のヒントに基づく場合の2回足し合わされている、ということになります。

一方、`eq_constraints` で実装されている一致制約は、そもそも左のヒント及び上のヒントから推定される要素が等しいとする制約ですので、2回の足し合わせ必要ありません。したがって、制約の大きさも遷移制約(ii-a')、(ii-b')、(ii-c') と比べて（大雑把には）半分程度である、と推測できます。

そこで、一致制約に2倍の重みを加えて、制約条件間の大きさを揃えることで、イジングマシン・量子アニーリングによる求解がしやすくなる、と考えられます。

 制約条件 `constraints` を `BinaryQuadraticModel` に与えることでの論理模型クラスとして定式化し、これを先ほど設定した `solver` に与えて実行します。

In [ ]:
from amplify import Solver, BinaryQuadraticModel

# 先にインスタンス化したクライアントで Amplify ソルバーを作成
solver = Solver(client)

model = BinaryQuadraticModel(constraints)
result = solver.solve(model)
if len(result.solutions) == 0:
    raise RuntimeError("No solution was found")

values = result.solutions[0].values

In [ ]:
solution = q_from_left_hints.decode(values)
plot_solution(left_hints, top_hints, solution)

<a id="4"></a>
# 発展：高度なチューニング

より大規模な問題を求解可能とするためには変数の数を減らす必要があります。前述の解法においても、[3.2.2\. 初期値制約](#3_2_2)や[3.2.3\. 遷移制約](#3_2_3) **(ii-d')**、[3.2.4\. 一致制約](#3_2_4)で、変数への代入を通じて、求解の対象となる変数の数を減らしました。

一致制約は、`equal_to` を用いて実装しましたが、ここでは、一致制約におけるいくつか要素の制約式を代入で解決することにより、変数の数をさらに減らすことを考えます。

より大規模な問題として、今回は 15x15 のピクロスパズルを用います。

In [ ]:
# ヒントをリストで表記
left_hints = [
    [],
    [5],
    [1, 1],
    [5, 1],
    [1, 1, 1],
    [1, 1, 5],
    [1, 2, 1],
    [9, 1],
    [1, 1, 1, 1],
    [5, 1, 1, 1],
    [1, 1, 1, 2],
    [1, 1, 5],
    [1, 2],
    [5],
    [],
]
top_hints = [
    [],
    [5],
    [2, 1],
    [5, 1, 1],
    [2, 1, 1, 1],
    [1, 1, 1, 5],
    [1, 1, 1, 1],
    [1, 9],
    [1, 2, 1],
    [5, 1, 1],
    [1, 1, 1],
    [1, 5],
    [1, 1],
    [5],
    [],
]

v_len, h_len = len(left_hints), len(top_hints)

plot_solution(left_hints, top_hints, np.zeros((v_len, h_len)))

5x5 の問題と同様に、変数を定義します（参考：[3.2.1\. 決定変数の定義](#3_2_1)）。

In [ ]:
gen = BinarySymbolGenerator()

# 左のヒントに基づくピクロス行列、マス (i,j) の色
q_from_left_hints = gen.array(v_len, h_len)
# 上のヒントに基づくピクロス行列、マス (i,j) の色
q_from_top_hints = gen.array(h_len, v_len)

# 左と上のヒントそれぞれに対応する変数を作成する
# (マスの数, 状態数)の二次元配列
p_from_left_hints = [gen.array(h_len, sum(hint) + len(hint) + 1) for hint in left_hints]
p_from_top_hints = [gen.array(v_len, sum(hint) + len(hint) + 1) for hint in top_hints]

定義した変数に対して、初期値制約、遷移制約 (ii-d') 及びマス色と状態をつなげる制約 (iii') を適用し、変数への代入を行います。

In [ ]:
# 左のヒントに対するピクロス列
for q, p, hint in zip(q_from_left_hints, p_from_left_hints, left_hints):
    # 初期値制約
    assign_init_value(p)
    # 遷移制約：(ii-d')
    assign_black_forward(p, hint)
    # マス色と状態をつなげる制約：(iii')
    assign_state_color_link(q, p, hint)

# 上のヒントに対するピクロス列
for q, p, hint in zip(q_from_top_hints, p_from_top_hints, top_hints):
    # 初期値制約
    assign_init_value(p)
    # 遷移制約：(ii-d')
    assign_black_forward(p, hint)
    # マス色と状態をつなげる制約：(iii')
    assign_state_color_link(q, p, hint)


`BinaryPoly`型の多項式では、`asdict()`関数を用いることにより多項式を一つずつの項に分割することができます。そこで、`asdict()`関数を使い、 項が2つ以下のみの一致制約を検出し、検出した制約を変数への代入に書き換えることで、変数の数と制約条件の数を減らします。

つまり、一致制約式である $q^{(0)}_{i,j} - q^{(1)}_{i,j}=0$ を満たす場合、左のヒントに基づく **塗り方0** の $i,j$ 成分は $q^{(0)}_{i,j}$ の代わりに $q^{(1)}_{i,j}$ であっても良い（逆もしかり）、ということになります。そうすることで、左のヒント及び上のヒントに基づく行列の $i,j$ 成分に対応する変数を2つから1つに減らすことができることが分かります。

ここで、すでに上記で、初期値制約、遷移制約 (ii-d') 及びマス色と状態をつなげる制約 (iii') による代入 (`assign_*`) が実施されているため、一致制約式である $q^{(0)}_{i,j} - q^{(1)}_{i,j}$ は必ずしも2項のみ有するとは限らない点に注意します。

そこで、まず、項が2つ以下のみの一致制約を検出し、2項以下からなる一致制約に対してのみ、代入する変数と等式で結ばれる多項式のペアを `substitute_dicts` に格納すると共に、`q_from_left_hints` と `q_from_top_hints` を一致させ、変数の数を減らします。

In [ ]:
substitute_dicts = dict()
for x in range(h_len):
    for y in range(v_len):
        c_dict = (q_from_left_hints[y, x] - q_from_top_hints[x, y]).asdict()
        for k, v in c_dict.items():
            if len(k) == 0:
                continue
            # substitute_dictsへの格納
            d = dict()
            d_max, d_min = 0, 0
            for k2, v2 in c_dict.items():
                if k == k2:
                    continue
                d[k2] = v2 / v * (-1)
                if len(k2) == 0:
                    d_max += d[k2]
                    d_min += d[k2]
                elif d[k2] > 0:
                    d_max += d[k2]
                else:
                    d_min += d[k2]
            # 多項式の係数 d が 0 <= d <= 1 を満たす場合のみ代入
            if d_min < 0 or d_max > 1:
                continue
            substitute_dicts[k] = d
            # q_from_left_hints と q_from_top_hints を一致させる
            if k in q_from_left_hints[y, x].asdict():
                q_from_left_hints[y, x] = q_from_top_hints[x, y]
            else:
                q_from_top_hints[x, y] = q_from_left_hints[y, x]
            break

次に、`p_from_left_hints` と `p_from_top_hints` に `substitute_dicts`（代入する変数と等式で結ばれる多項式のペア）の内容を反映させます。

In [ ]:
from amplify import BinaryPoly


def substitute_according_to_dicts(p, substitute_dicts):
    for j in range(p.shape[0]):
        for k in range(p.shape[1]):
            d = p[j, k].asdict()
            if len(d) == 0:
                continue
            key, _ = d.popitem()
            if key in substitute_dicts:
                p[j, k] = BinaryPoly(substitute_dicts[key])


for p in p_from_left_hints:
    substitute_according_to_dicts(p, substitute_dicts)

for p in p_from_top_hints:
    substitute_according_to_dicts(p, substitute_dicts)

上記により変数の数を減らした後は、通常の定式化と同様に制約条件を作成します。

In [ ]:
# 遷移制約：(ii-a')、(ii-b')、(ii-c')
state_constraints = []
white_constraints = []
last_black_constraints = []
for p, hint in zip(p_from_left_hints, left_hints):
    state_constraints += make_state_constraint(p)
    white_constraints += make_white_constraint_forward(p, hint)
    last_black_constraints += make_white_constraint_backward(p, hint)
left_constraints = (
    sum(state_constraints) + sum(white_constraints) + sum(last_black_constraints)
)

state_constraints = []
white_constraints = []
last_black_constraints = []
for p, hint in zip(p_from_top_hints, top_hints):
    state_constraints += make_state_constraint(p)
    white_constraints += make_white_constraint_forward(p, hint)
    last_black_constraints += make_white_constraint_backward(p, hint)
top_constraints = (
    sum(state_constraints) + sum(white_constraints) + sum(last_black_constraints)
)

# 一致制約
eq_constraints = equal_constraints(q_from_left_hints, q_from_top_hints, v_len, h_len)

constraints = left_constraints + top_constraints + eq_constraints * 2

問題サイズが増加したので、先に作成したクライアントのタイムアウトを大きく設定し、その後ソルバーを再度作成します。

In [ ]:
client.parameters.timeout = 10000  # タイムアウト10秒
solver = Solver(client)

 制約条件 `constraints` を `BinaryQuadraticModel` に与えることでの論理模型クラスとして定式化し、これを先ほど設定した `solver` に与えて実行します。

In [ ]:
model = BinaryQuadraticModel(constraints)
result = solver.solve(model)
if len(result.solutions) == 0:
    raise RuntimeError("No solution was found")

values = result.solutions[0].values

変数配列 `q_from_left_hints` の `decode` メンバ関数に解である`values` を与え、`solution` に結果を代入します。その後、`plot_solution` 関数を用いて解答を出力します。

In [ ]:
solution = q_from_left_hints.decode(values)
plot_solution(left_hints, top_hints, solution)